In [73]:
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from urllib.request import urlopen
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd
import os
import pickle
import pdb
import requests
import re
from IPython.display import display, HTML
from bs4 import BeautifulSoup, Comment
%matplotlib inline

In [39]:
NBA_URL = 'https://www.basketball-reference.com'
TEAMS = [
    #Initials, Name
    ('ATL', 'Hawks'),
    ('BOS', 'Celtics'),
    ('BRK', 'Nets'),
    ('CHI', 'Bulls'),
    ('CHO', 'Hornets'),
    ('CLE', 'Cavaliers'),
    ('DAL', 'Mavericks'),
    ('DEN', 'Nuggets'),
    ('DET', 'Pistons'),
    ('GSW', 'Warriors'),
    ('HOU', 'Rockets'),
    ('IND', 'Pacers'),
    ('LAC', 'Clippers'),
    ('LAL', 'Lakers'),
    ('MEM', 'Grizzlies'),
    ('MIA', 'Heat'),
    ('MIL', 'Bucks'),
    ('MIN', 'Timberwolves'),
    ('NOP', 'Pelicans'),
    ('NYK', 'Knicks'),
    ('OKC', 'Thunder'),
    ('ORL', 'Magic'),
    ('PHI', '76ers'),
    ('PHO', 'Suns'),
    ('POR', 'Trailblazers'),
    ('SAC', 'Kings'),
    ('SAS', 'Spurs'),
    ('TOR', 'Raptors'),
    ('UTA', 'Jazz'),
    ('WAS', 'Wizards')
]
TEAM_NAMES = dict()

def get_season_data_per_poss(year):
    with (open("Data/" + str(year) + "/all_team-stats-per_poss.pkl", "rb")) as openfile:
        data = pickle.load(openfile)
    with (open("Data/" + str(year) + "/all_opponent-stats-per_poss.pkl", "rb")) as openfile:
        opp_data = pickle.load(openfile)
    return data, opp_data

def get_season_data(year):
    with (open("Data/" + str(year) + "/all_team-stats-base.pkl", "rb")) as openfile:
        data = pickle.load(openfile)
    with (open("Data/" + str(year) + "/all_opponent-stats-base.pkl", "rb")) as openfile:
        opp_data = pickle.load(openfile)
    return data, opp_data

    #ratio, wins = convert_data_to_ratio(data)
def convert_data_to_ratio(data):
    wins = data[data.columns[-1]]
    data = data.drop(labels='Win', axis=1)
    columns = data.columns
    for column in columns:
        if '%' in column:
            data = data.drop(column, axis=1)
    away = data[data.columns[:len(data.columns)//2]]
    home = data[data.columns[len(data.columns)//2:]]
    ratio = []
    for index, row in away.iterrows():
        away_row = np.array(away.iloc[index], dtype=np.float)
        home_row = np.array(home.iloc[index], dtype=np.float)
        home_row[home_row == 0] = 1
        ratio.append(away_row / home_row)
    assert(len(ratio) == len(wins))
    return ratio, wins
def get_team_names(year):
    rtn = []
    standings_soup = "https://www.basketball-reference.com/leagues/NBA_" + str(year) + "_standings.html"
    standings_soup = BeautifulSoup(urlopen(standings_soup), 'html.parser')
    comment = standings_soup.find("div",  id='all_expanded_standings')\
            .findAll(text=lambda text:isinstance(text, Comment))
    extracted_comment = comment[0].extract()
    standings_soup = BeautifulSoup(extracted_comment)
    rows = standings_soup.find('tbody').findAll("tr")
    for row in rows:
        rtn.append(row.find("td", attrs={'data-stat': "team_name"}).getText())
    return sorted(rtn)
    
    
# training data: previous year's stats + current season stats
def get_team_cumulative_stats(year):
    team_stats = dict()
    for initials, name in TEAMS:
        with (open("Data/" + str(year-1) + "/" + str(initials) + \
                   "_season_game_basic_cumulative_stats.pkl", "rb")) as openfile:    
            team_stats[initials] = pickle.load(openfile)
    return team_stats
def get_total_games(team_stats):
    num_games = 0
    for key in team_stats:
        num_games +=  team_stats[key].shape[0]
    return num_games
    
#create training data
def get_training_data(year):
    training_data = []
    results = []
    team_names = get_team_names(year)
    assert(team_names == get_team_names(year-1))
    
    team_cumulative_stats = get_team_cumulative_stats(year)
    season_data, season_opp_data = get_season_data_per_poss(year-1)
    season_data_dict = convert_season_data_to_dict(season_data, team_names)
    season_opp_data_dict = convert_season_data_to_dict(season_opp_data, team_names)
    for key in season_data_dict.keys():
        assert(key in season_opp_data_dict)
        season_data_dict[key] = np.concatenate((season_data_dict[key], season_opp_data_dict[key]), axis=None)
    sample = "https://www.basketball-reference.com/leagues/NBA_2019_games.html"
    year_soup = BeautifulSoup(urlopen(sample), 'html.parser')
    months = year_soup.find("div", attrs={'class': 'filter'})
    months = months.findAll("a")
    months = [NBA_URL + month['href'] for month in months]
    
    for month in months:
        month_soup = BeautifulSoup(urlopen(month), 'html.parser')
        games = month_soup.find("table", attrs={'id': 'schedule'}).find('tbody').findAll("tr")
        for game in games:
            game_data = []
            win = 0
            if game.find('td') and game.find('td', attrs={'data-stat':'visitor_pts'}) \
                and game.find('td', attrs={'data-stat':'visitor_pts'}).getText():
                away = game.find('td', attrs={'data-stat':'visitor_team_name'}).getText()
                home = game.find('td', attrs={'data-stat':'home_team_name'}).getText()
                away_pts = game.find('td', attrs={'data-stat':'visitor_pts'}).getText()
                away_pts = int(away_pts)
                home_pts = game.find('td', attrs={'data-stat':'home_pts'}).getText()
                home_pts = int(home_pts)
                game_data.extend(season_data_dict[away])
                game_data.extend(season_data_dict[home])
                assert(home_pts != away_pts)
                if home_pts > away_pts:
                    win = 1
                results.append(win)
                training_data.append(game_data)
    assert(len(training_data) == len(results))
    return training_data, results
            
                
                
                
def convert_season_data_to_dict(pandas_frame, team_names):
    pandas_frame.drop([column for column in list(pandas_frame) if '%' in column], inplace=True, axis=1)
    pandas_frame.drop([column for column in list(pandas_frame) if 'FG' in column], inplace=True, axis=1)
    pandas_frame.drop([column for column in list(pandas_frame) if 'PTS' in column], inplace=True, axis=1)
    output = dict()
    count = 0
    for team in pandas_frame['Team']:
        for team_name in team_names:
            if team_name in team:
                row = pandas_frame.iloc[count, 3:]
                output[team_name] = row.values
        count += 1
    return output


In [40]:
#get_training_data()
data, results = get_training_data(2019)

In [41]:
data = np.array(data).astype(float)
results = np.array(results).astype(float)

In [81]:
kFold = KFold(n_splits=5, shuffle=True, random_state=None)
regression_model = LinearRegression()
for train_index, test_index in kFold.split(data):
    
    train_data, train_result = data[train_index], results[train_index]
    test_data, test_result = data[test_index], results[test_index]
    regression_model.fit(train_data, train_result)
    test_prediction = regression_model.predict(test_data)

    total = len(test_prediction)
    correct = 0
    for i in range(len(test_prediction)):
        if test_prediction[i] > 0.5:
            test_prediction[i] = 1
        else:
            test_prediction[i] = 0
        if test_prediction[i] == test_result[i]:
            correct += 1
    print(float(correct)/ total)
    rmse = mean_squared_error(test_result, test_prediction)
    r2 = r2_score(test_result, test_prediction)
    
    
    polynomial_features = PolynomialFeatures(degree=2)
    train_data_poly = polynomial_features.fit_transform(train_data)
    test_data_poly = polynomial_features.fit_transform(test_data)
    polynomial_model = LinearRegression()
    polynomial_model.fit(train_data_poly,train_result)
    polynomial_test_prediction = polynomial_model.predict(test_data_poly)
    total = len(polynomial_test_prediction)
    correct = 0
    for i in range(len(polynomial_test_prediction)):
        if polynomial_test_prediction[i] > 0.5:
            polynomial_test_prediction[i] = 1
        else:
            polynomial_test_prediction[i] = 0
        if polynomial_test_prediction[i] == test_result[i]:
            correct += 1
    print(float(correct)/ total)
    
    rsme_polynomial = mean_squared_error(test_result, polynomial_test_prediction)
    r2_polynomial = r2_score(polynomial_test_prediction, test_result)
    
# model evaluation
    logistic_model = LogisticRegression()
    logistic_model.fit(train_data, train_result)
    logistic_test_prediction = logistic_model.predict(test_data)
    test_prediction = logistic_test_prediction
    
    total = len(test_prediction)
    correct = 0
    for i in range(len(test_prediction)):
        if test_prediction[i] > 0.5:
            test_prediction[i] = 1
        else:
            test_prediction[i] = 0
        if test_prediction[i] == test_result[i]:
            correct += 1
    print(float(correct)/ total)
    rmse_logistic = mean_squared_error(test_result, test_prediction)
    r2_logistic = r2_score(test_result, test_prediction)


    

# printing values
    #print('Slope:' ,regression_model.coef_)
    #print('Intercept:', regression_model.intercept_)
    print('Root mean squared error: ', rmse)
    print('R2 score: ', r2)
    
    print('Polynomial Root mean squared error: ', rsme_polynomial)
    print('Polynomial R2 score: ', r2_polynomial)
        
    print('Logistic Root mean squared error: ', rmse_logistic)
    print('Logistic R2 score: ', r2_logistic)
    break


0.6395348837209303
0.5736434108527132
0.6472868217054264
Root mean squared error:  0.36046511627906974
R2 score:  -0.47383292383292375
Polynomial Root mean squared error:  0.4263565891472868
Polynomial R2 score:  -0.7614200595829197
Logistic Root mean squared error:  0.35271317829457366
Logistic R2 score:  -0.4421375921375921


/home/jlee/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
